In [ ]:
!pip install spark-nlp
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=223ca7dbb0342ffdba8478a5edd5f5724f6272f781c992877199f90ec6fd10b4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pkg_resources

# Thay thế "spark-nlp" bằng tên gói của Spark NLP (nếu khác)
package_name = "spark-nlp"

try:
    version = pkg_resources.get_distribution(package_name).version
    print(f"Phiên bản của {package_name} là: {version}")
except pkg_resources.DistributionNotFound:
    print(f"{package_name} không được cài đặt.")

Phiên bản của spark-nlp là: 5.0.1


In [ ]:
from pyspark.sql import SparkSession, DataFrame

In [ ]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, Tokenizer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()
data = spark.read.csv("/content/Amazon_product_review.csv", header=True, inferSchema=True).na.drop()
(train_data, test_data) = data.randomSplit([0.7, 0.3], seed=42)


In [ ]:

tokenizer = Tokenizer(inputCol="short_review", outputCol="words")
hashing_tf = HashingTF(inputCol="words", outputCol="raw_features", numFeatures=1000)
idf = IDF(inputCol="raw_features", outputCol="features")
indexer = StringIndexer(inputCol="Sentiment", outputCol="label", handleInvalid="keep")
rf_classifier = RandomForestClassifier(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, indexer, rf_classifier])

In [ ]:
model = pipeline.fit(train_data)
predictions = model.transform(test_data)

In [ ]:
preds.show()

+--------------------+---------+--------------------+--------------------+--------------------+
|        short_review|Sentiment|            document|               token|               label|
+--------------------+---------+--------------------+--------------------+--------------------+
|"Fast service, th...| positive|[{document, 0, 39...|[{token, 0, 0, ",...|[{category, 0, 39...|
|"Really impressed...| positive|[{document, 0, 36...|[{token, 0, 0, ",...|[{category, 0, 36...|
|"Super extended l...| negative|[{document, 0, 19...|[{token, 0, 0, ",...|[{category, 0, 19...|
|"The armband is g...| positive|[{document, 0, 35...|[{token, 0, 0, ",...|[{category, 0, 35...|
|&34;BUYER BEWARE ...| negative|[{document, 0, 35...|[{token, 0, 8, &3...|[{category, 0, 35...|
|&34;you get what ...| negative|[{document, 0, 25...|[{token, 0, 6, &3...|[{category, 0, 25...|
|*** UPDATE Novemb...| positive|[{document, 0, 10...|[{token, 0, 2, **...|[{category, 0, 10...|
|***ORIGINAL REVIE...| positive|[{docume

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Step 9: Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)

print("F1 Score: {:.4f}".format(f1_score))


F1 Score: 0.6284
